# Spot segmentation notebook 
# Elizabeth Finn
# May 24, 2023

## Will:

1. Find nuclei from nuclear segmentation results
2. Load 10 at random according to your wells of interest
3. Allow iterative optimization of parameters
4. Segment spots using a Laplacian of the Gaussian for selected nuclei and parameters
5. Output to a config file for bulk analysis
6. Format, write, and run shell scripts for bulk analysis

# First, set basic parameters and load in necessary tools.

In [1]:
ANALYSIS_PREFIX = "2023-07-18"
MICROSCOPE = "IMX"
USER = "butlerm"
DAPI_CHANNEL = 1

In [2]:
# Set filetype environment variable here, before anything is loaded... "LSM" is LSM, "IMX" is default and for ImageXpress Micro
import os
os.environ["FILE_TYPE"] = MICROSCOPE
os.environ["USER"] = USER

In [3]:
#import some tools that we will use:
# Utilities
import json
import random
import numpy
import csv
from pathlib import Path

#Sci-kit image and matplotlib
import skimage.io
import skimage.measure
import skimage.feature
import skimage.segmentation
from skimage.morphology import disk
import scipy.ndimage
import matplotlib.pyplot as plt

#Our bespoke tools
from models.swarm_job import SwarmJob, RunStrategy
from models.image_filename_glob import ImageFilenameGlob
from models.generate_spot_positions_config import GenerateSpotPositionsConfig
from generate_spot_positions import GenerateSpotPositionsJob
from generate_all_spot_positions import GenerateAllSpotPositionsJob
from generate_all_spot_result_lines import GenerateAllSpotResultLinesJob
from generate_spot_results_file import GenerateSpotResultsFileJob

# FROM HERE, YOU MUST RUN ITERATIVELY, MULTIPLE TIMES PER CHANNEL, AS YOU OPTIMIZE. 

## The overall process looks like this:
### For each channel:
#### Do the following until selecting a new set of random nuclei doesn't substantially alter the segmentation (i.e. until it looks good every time you re-cycle through positions and samples). Be sure to test at least five different positions (wells or coverslips) and at least three different random subsets of nuclei per position:
        1. Select a random set of 10 nuclei (first chunk)
        2. Choose parameters (second chunk)
        3. Plot nuclei and spot results (third chunk)
#### Once that is done, and you're comfortable with the parameters for the channel of interest, add that channel's parameters to the config file (fourth chunk). Note that you don't need to change anything, it will automatically detect if there's already a config file and append/replace appropriately if so.
            

In [ ]:
image_folder = "/s/finn-lab/MB_0718/crops/"
image_path = Path(image_folder)

position_ID = "C07"
channel_ID = 2

image_filename_glob = image_path.rglob(str(ImageFilenameGlob(position = position_ID,
                                                             c = channel_ID, 
                                                             suffix="_maximum_projection_nucleus_???", 
                                                             extension="npy")))

cells_list = ([str(image_file_path) for image_file_path in image_filename_glob])
files_count = len(cells_list)
if files_count == 0:
    print("I didn't find any nuclei, check another field.")
else: 
    images_selected = random.sample(cells_list, min(10, files_count))
    if files_count < 10:
        print("I only found ", files_count,  "nuclei")

for image_selected in images_selected:
    print(image_selected)

#### Set the parameters and display nuclei with spots highlighted in the next two chunks. With each batch of 10 nuclei, run the next two chunks as many times as you need.

In [ ]:
# Threshold is applied to the laplacian of the gaussian, it is fairly cryptic, so trial and error works best to find a good value.
LoG_threshold = 0.8

# Fold change over local (radius *2) average
local_contrast_threshold = 1.2

# Peak radius is applied to pre-filter the spots; anything smaller than this will get filtered out, so err on the small side. 
# It ought to be near the actual radius of the visible spot in pixels.
peak_radius = 2.0

# Because images are normalized to 0-1 range in the cell, your spot maxima are set to 1. So this ought to be the inverse of your overall signal:noise ratio
# Counterintuitively, better FISH -> lower threshold here -> less filtering taking place on the actual image
global_contrast_threshold = 0.3

segmenters = [GenerateSpotPositionsJob(image, "no_output", image_folder, LoG_threshold, local_contrast_threshold, peak_radius, global_contrast_threshold) for image in images_selected]

In [ ]:
fig, axs = plt.subplots(2,5)

for i in range(0,10):
    ax = axs[i//5, i%5]
    ax.imshow(segmenters[i].image)
    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True)
    for spot in segmenters[i].spots:
        circle=plt.Circle((spot[4], spot[3]), 7, color='r', alpha=0.3)
        ax.add_patch(circle)
        #ax.text(x = spot[1]-20, y = spot[0]-15, s = "%.2f, %.2f"%(spot[1], spot[0]), bbox=dict(fill=True))

fig.set_size_inches(15,7)
fig.canvas.draw()

#### Output the selected parameters into a config file. This chunk can be run multiple times per channel but must be run at least once per channel -- if you have channels without parameters, they will not be segmented. Code will:
1. If a config file exists, open it and read it into a table.
2. Alter that table as necessary, or generate a new one with the current channel and parameters.
3. Overwrite the config file (or write a new config file) with the updated table.

In [ ]:
config_file = "/hpc-prj/finn/Results/configs/MBspotsconfig.json"

config_file_path = Path(config_file)

existing_generate_spot_positions_configs = []
if config_file_path.exists():
    with open(config_file_path) as json_file:
        existing_generate_spot_positions_configs_json_params = json.load(json_file)
        existing_generate_spot_positions_configs = [
            GenerateSpotPositionsConfig.from_json_params(json_params)
            for json_params
            in existing_generate_spot_positions_configs_json_params
        ]

current_generate_spot_positions_config = GenerateSpotPositionsConfig(
    channel=channel_ID,
    LoG_threshold=LoG_threshold,
    local_contrast_threshold=local_contrast_threshold,
    peak_radius=peak_radius,
    global_contrast_threshold=global_contrast_threshold
)

updated_generate_spot_positions_configs = [
    current_generate_spot_positions_config,
    *(
        existing_generate_spot_positions_config
        for existing_generate_spot_positions_config
        in existing_generate_spot_positions_configs
        if existing_generate_spot_positions_config.channel != current_generate_spot_positions_config.channel
    )
]

updated_generate_spot_positions_configs_json_params = [config.to_json_params() for config in updated_generate_spot_positions_configs]
with open(config_file_path, 'w') as write_file:
    json.dump(updated_generate_spot_positions_configs_json_params, write_file)


## We're ready to run in batch mode! Congratulations! 
### Here's what the following chunks do:
1. Load in relevant parameters and folders. (Parameters: local or parallel processing via RunStrategy and location of config file.)
2. Segment all the spots
3. Determine centers of gravity, size, shape, and intensity for all spots.
4. Collate everything into a csv file.

### Each chunk will generate a shell script, run it, and email you approximately when it is completed. However, it is generally faster and more reliable to monitor the shell script using the "squeue" command on the command line. Do not run chunk two until the shell script in chunk one has finished running, since the files will not be all generated and will therefore get lost.

In [4]:
# Find your files!
SwarmJob.run_strategy = RunStrategy.SWARM

source_cropped_images = "/s/finn-lab/MB_0718/crops/"
source_nuclear_masks = "/s/finn-lab/MB_0718/masks/"
source_distance_transforms = "/s/finn-lab/MB_0718/dts/"

results_directory = Path("/hpc-prj/finn/Results")
scratch_directory = Path("/s/finn-lab/MB_0718")
spot_positions_directory = scratch_directory / "spot_positions/"
spot_result_lines_directory = scratch_directory / "spot_result_lines/"

log_directory = results_directory / "logs/"

In [ ]:
# Run the spot segmentation! 

GenerateAllSpotPositionsJob(source_cropped_images, spot_positions_directory, log_directory, DAPI_CHANNEL, config_file).run()

In [ ]:
# From spot segmentations, find x, y, z, and r positions!
GenerateAllSpotResultLinesJob(spot_positions_directory, source_cropped_images, 
                              source_distance_transforms, source_nuclear_masks, 
                              spot_result_lines_directory, log_directory).run()

Generating file dictionary (this can take a while)
running generate_all_spot_result_lines_20230721143357 in parallel mode
['sbatch', '--wait', PosixPath('/s/finn-lab/MB_0718/spot_result_lines/generate_all_spot_result_lines_20230721143357.sh')]


Submitted batch job 1911659


In [6]:
# Find and merge all spot positions into a single file!
GenerateSpotResultsFileJob(spot_result_lines_directory, results_directory, ANALYSIS_PREFIX).run()